### There are CSV files containing Citi Bike data

Renamed columns to a consistent snake_case schema

Converted start_time/stop_time to datetime64[ns] and computed trip_duration as timedelta64[ns]

Cleaned categorical variables (mapped gender codes to labels, preserved true missing values)

Added boolean flags for missing user_type, birth_year, and gender, plus a combined “bad/missing demographics” flag

Created an age feature (ride-year minus birth_year) and flagged implausible birth years (e.g., <= 1900)

In [1]:
# ✅ import all the cool stuff we are working with
import pandas as pd
import glob
import numpy as np

In [2]:
# ✅ Loading and concatenating the Citi Bike CSVs
bike_files = glob.glob('data/JC-*-citibike-tripdata.csv')

df_list = []
for filename in bike_files:
    data = pd.read_csv(filename)
    df_list.append(data)
    bikes = pd.concat(df_list)

bikes = bikes.reset_index().drop(columns='index')

# ✅ Getting a sense of the data
bikes.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


In [3]:
# ✅ check info to understand how much data I have to deal with
# bikes.info()

In [4]:
# ✅ show how much missing values there are
bikes.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

In [5]:
# User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
# Gender (Zero=unknown; 1=male; 2=female)

# ✅ rename columns to snake_type
bikes.columns = ['trip_duration', 'start_time', 'stop_time', 'start_station_id', 'start_station_name', 'start_station_lat', 'start_station_long', 'end_station_id','end_station_name' , 'end_station_lat', 'end_station_long', 'bike_id', 'user_type', 'birth_year', 'gender']

# ✅ replace 0, 1, 2 in gender column with the according labels
bikes.gender = bikes.gender.replace(1, 'male', regex=False)
bikes.gender = bikes.gender.replace(2, 'female', regex=False)
bikes.gender = bikes.gender.replace(0, np.nan, regex=False)

# ✅ change datatype to datetime
bikes.start_time = bikes.start_time.astype('datetime64[ns]')
bikes.stop_time = pd.to_datetime(bikes.stop_time)

bikes.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,birth_year,gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,female
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,male
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,female
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,male
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,NaN


In [6]:
# ✅ make trip_duration an actual timedelta column for more comfortable data analysis
bikes.trip_duration = bikes.stop_time - bikes.start_time

bikes.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,birth_year,gender
0,0 days 00:06:02,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,female
1,0 days 00:03:20,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,male
2,0 days 00:03:22,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,female
3,0 days 00:04:08,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,male
4,0 days 00:15:04,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,NaN


In [7]:
# ✅ check that all our columns are according dtype
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247584 entries, 0 to 247583
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   trip_duration       247584 non-null  timedelta64[ns]
 1   start_time          247584 non-null  datetime64[ns] 
 2   stop_time           247584 non-null  datetime64[ns] 
 3   start_station_id    247584 non-null  int64          
 4   start_station_name  247584 non-null  object         
 5   start_station_lat   247584 non-null  float64        
 6   start_station_long  247584 non-null  float64        
 7   end_station_id      247584 non-null  int64          
 8   end_station_name    247584 non-null  object         
 9   end_station_lat     247584 non-null  float64        
 10  end_station_long    247584 non-null  float64        
 11  bike_id             247584 non-null  int64          
 12  user_type           247204 non-null  object         
 13  birth_year    

In [8]:
# ✅ create Boolean columns for more comfortable analysis of columns where something is missing
bikes['missing_user_type'] = bikes.user_type.isna()
bikes['missing_birth_year'] = bikes.birth_year.isna()
bikes['missing_gender'] = bikes.gender.isna()

# bikes.head(15)

In [9]:
# ✅ create one Boolean column to show if there's anything missing (again for comfortable analysis)
bikes['has_missing'] = (
    (bikes.missing_user_type == True) |
    (bikes.missing_birth_year == True) |
    (bikes.missing_gender == True)
)

# ✅ another Boolean column to show if there's any implausible data (can add other things if found later as well). Usually 1900 is a placeholder year of birth
bikes['has_bad'] = (
    (bikes.birth_year <= 1900) |
    (bikes.trip_duration <= '00:00:00') |
    (bikes.trip_duration >= '00:45:00')
)

# ✅ calculating age in a separate column
bikes['age'] = 2016 - bikes.birth_year

# ✅ check final result
bikes

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,...,bike_id,user_type,birth_year,gender,missing_user_type,missing_birth_year,missing_gender,has_missing,has_bad,age
0,0 days 00:06:02,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,...,24647,Subscriber,1964.0,female,False,False,False,False,False,52.0
1,0 days 00:03:20,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,...,24605,Subscriber,1962.0,male,False,False,False,False,False,54.0
2,0 days 00:03:22,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,...,24689,Subscriber,1962.0,female,False,False,False,False,False,54.0
3,0 days 00:04:08,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,...,24693,Subscriber,1984.0,male,False,False,False,False,False,32.0
4,0 days 00:15:04,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,...,24573,Customer,NaN,NaN,False,True,True,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247579,0 days 00:09:17,2016-12-31 23:10:16,2016-12-31 23:19:33,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,...,24465,Subscriber,1981.0,female,False,False,False,False,False,35.0
247580,0 days 00:45:50,2016-12-31 23:29:39,2017-01-01 00:15:29,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,...,24389,Customer,NaN,NaN,False,True,True,True,True,NaN
247581,0 days 00:02:54,2016-12-31 23:44:37,2016-12-31 23:47:31,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,...,24641,Subscriber,1978.0,male,False,False,False,False,False,38.0
247582,0 days 00:40:24,2016-12-31 23:44:50,2017-01-01 00:25:14,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,...,26219,Subscriber,1960.0,female,False,False,False,False,False,56.0


In [10]:
# ✅ last bit with descriptive statistics
bikes.describe()

,trip_duration,start_time,stop_time,start_station_id,start_station_lat,start_station_long,end_station_id,end_station_lat,end_station_long,bike_id,birth_year,age
count,247584,247584,247584,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,228585.000000
mean,0 days 00:14:46.129737785,2016-07-29 05:55:07.541335040,2016-07-29 06:09:53.671073536,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,36.664724
min,-1 days +23:05:29,2016-01-01 00:02:52,2016-01-01 00:08:54,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,16.000000
25%,0 days 00:04:09,2016-05-27 07:46:06,2016-05-27 07:54:40.249999872,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,30.000000
50%,0 days 00:06:30,2016-08-10 09:23:50,2016-08-10 09:34:32.500000,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,35.000000
75%,0 days 00:11:07,2016-10-05 17:25:05.500000,2016-10-05 17:33:00.750000128,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,42.000000
max,189 days 00:03:28,2016-12-31 23:44:50,2017-01-18 14:26:46,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,116.000000
std,0 days 09:58:58.104602902,NaN,NaN,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,9.596809


In [11]:
# ✅ check all the bad data
has_bad_data = bikes[bikes.has_bad == True]
has_bad_data

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,...,bike_id,user_type,birth_year,gender,missing_user_type,missing_birth_year,missing_gender,has_missing,has_bad,age
53,0 days 00:51:31,2016-01-01 11:07:15,2016-01-01 11:58:46,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,...,24444,Customer,NaN,NaN,False,True,True,True,True,NaN
68,0 days 00:49:17,2016-01-01 13:09:23,2016-01-01 13:58:40,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,...,24609,Subscriber,1978.0,male,False,False,False,False,True,38.0
99,0 days 01:09:48,2016-01-01 15:24:16,2016-01-01 16:34:04,3213,Van Vorst Park,40.718489,-74.047727,3213,Van Vorst Park,40.718489,...,24427,Subscriber,1968.0,female,False,False,False,False,True,48.0
104,0 days 00:49:40,2016-01-01 15:41:34,2016-01-01 16:31:14,3187,Warren St,40.721124,-74.038051,3187,Warren St,40.721124,...,24605,Subscriber,1992.0,male,False,False,False,False,True,24.0
229,0 days 01:05:40,2016-01-02 12:32:42,2016-01-02 13:38:22,3184,Paulus Hook,40.714145,-74.033552,3184,Paulus Hook,40.714145,...,24656,Subscriber,1975.0,female,False,False,False,False,True,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247424,0 days 01:57:55,2016-12-31 13:41:43,2016-12-31 15:39:38,3185,City Hall,40.717732,-74.043845,3213,Van Vorst Park,40.718489,...,24513,Customer,NaN,NaN,False,True,True,True,True,NaN
247441,0 days 00:46:47,2016-12-31 14:30:46,2016-12-31 15:17:33,3183,Exchange Place,40.716247,-74.033459,3272,Jersey & 3rd,40.723332,...,26167,Subscriber,1954.0,female,False,False,False,False,True,62.0
247443,0 days 22:07:49,2016-12-31 14:35:25,2017-01-01 12:43:14,3275,Columbus Drive,40.718355,-74.038914,3199,Newport Pkwy,40.728745,...,26217,Customer,NaN,NaN,False,True,True,True,True,NaN
247508,0 days 01:12:46,2016-12-31 16:51:39,2016-12-31 18:04:25,3198,Heights Elevator,40.748716,-74.040443,3202,Newport PATH,40.727224,...,26178,Subscriber,1954.0,female,False,False,False,False,True,62.0


In [12]:
# ✅ and all the data that somewhere is empty or incomplete
has_empty = bikes[bikes.has_missing == True]
has_empty

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_lat,start_station_long,end_station_id,end_station_name,end_station_lat,...,bike_id,user_type,birth_year,gender,missing_user_type,missing_birth_year,missing_gender,has_missing,has_bad,age
4,0 days 00:15:04,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,...,24573,Customer,NaN,NaN,False,True,True,True,False,NaN
5,0 days 00:14:43,2016-01-01 01:03:28,2016-01-01 01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,...,24442,Customer,NaN,NaN,False,True,True,True,False,NaN
22,0 days 00:16:29,2016-01-01 03:16:33,2016-01-01 03:33:02,3196,Riverview Park,40.744319,-74.043991,3209,Brunswick St,40.724176,...,24662,Customer,NaN,NaN,False,True,True,True,False,NaN
53,0 days 00:51:31,2016-01-01 11:07:15,2016-01-01 11:58:46,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,...,24444,Customer,NaN,NaN,False,True,True,True,True,NaN
57,0 days 00:13:09,2016-01-01 11:50:30,2016-01-01 12:03:39,3210,Pershing Field,40.742677,-74.051789,3195,Sip Ave,40.730743,...,24573,Customer,NaN,NaN,False,True,True,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247442,0 days 00:41:25,2016-12-31 14:35:19,2016-12-31 15:16:44,3275,Columbus Drive,40.718355,-74.038914,3199,Newport Pkwy,40.728745,...,24627,Customer,NaN,NaN,False,True,True,True,False,NaN
247443,0 days 22:07:49,2016-12-31 14:35:25,2017-01-01 12:43:14,3275,Columbus Drive,40.718355,-74.038914,3199,Newport Pkwy,40.728745,...,26217,Customer,NaN,NaN,False,True,True,True,True,NaN
247537,0 days 00:03:45,2016-12-31 18:01:38,2016-12-31 18:05:23,3194,McGinley Square,40.725340,-74.067622,3195,Sip Ave,40.730743,...,24716,Subscriber,NaN,NaN,False,True,True,True,False,NaN
247561,0 days 00:07:21,2016-12-31 20:12:26,2016-12-31 20:19:47,3195,Sip Ave,40.730743,-74.063784,3193,Lincoln Park,40.724605,...,24417,Subscriber,NaN,NaN,False,True,True,True,False,NaN


In [13]:
# ✅ see the clean data without any missing data and outliers
clean_df = bikes[(~bikes.has_missing & ~bikes.has_bad)]

clean_df.describe()

,trip_duration,start_time,stop_time,start_station_id,start_station_lat,start_station_long,end_station_id,end_station_lat,end_station_long,bike_id,birth_year,age
count,225097,225097,225097,225097.000000,225097.000000,225097.000000,225097.000000,225097.000000,225097.000000,225097.000000,225097.000000,225097.000000
mean,0 days 00:08:21.385549340,2016-07-29 14:47:54.159202304,2016-07-29 14:56:15.544751872,3207.190998,40.723291,-74.046609,3204.029894,40.722723,-74.046005,24938.594246,1979.329942,36.670058
min,0 days 00:01:01,2016-01-01 00:02:52,2016-01-01 00:08:54,3183.000000,40.692640,-74.096937,173.000000,40.692216,-74.096937,14552.000000,1934.000000,16.000000
25%,0 days 00:04:01,2016-05-26 17:32:53,2016-05-26 17:45:01,3186.000000,40.717732,-74.050656,3186.000000,40.717732,-74.050444,24491.000000,1974.000000,30.000000
50%,0 days 00:06:07,2016-08-10 21:24:59,2016-08-10 21:38:06,3202.000000,40.721525,-74.044247,3198.000000,40.721124,-74.043845,24611.000000,1981.000000,35.000000
75%,0 days 00:10:01,2016-10-06 21:22:10,2016-10-06 21:29:53,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.038051,24719.000000,1986.000000,42.000000
max,0 days 00:44:59,2016-12-31 23:44:50,2017-01-01 00:25:14,3426.000000,40.752559,-74.032108,3426.000000,40.760875,-73.971212,27274.000000,2000.000000,82.000000
std,0 days 00:06:53.652273025,NaN,NaN,27.017247,0.008129,0.011173,50.726917,0.007865,0.011227,750.121291,9.590069,9.590069


In [14]:
bikes.to_csv('citibike_2016_cleaned.csv')